In [ ]:
library(tidyverse)
library(dplyr)

In [ ]:
mat <- read.csv("../input/aml-fab-with-names/GSE147515_FAB_mat.csv")
head(mat)

In [ ]:
lbl <- read.csv("../input/aml-fab-with-names/GSE147515_FAB_lbl.csv")
head(lbl)

In [ ]:
save_pheatmap_png <- function(x, filename, width=2500, height=1000, res = 200) {
  png(filename, width = width, height = height, res = res)
  grid::grid.newpage()
  grid::grid.draw(x$gtable)
  dev.off()
}

# M0

In [ ]:
m0.geneStr <-
"
NFKB1
EGFR
RXRA
JUN
CREBBP
CDK4
RAC1
EZH2
HSP90AB1
NCK1
PML
CREB1
SIRT1
CRK
SMAD4
THBS1
SMAD2
ESR1
EP300
APP
SRC
BLM
AKT1
RAD51
RAD52
CDKN1A
ABL1
RARA
EGR1
TP53
MDM2
CAMP
JUNB
MAPK1
JUND
GRB2
TGFB1
MYC
MAPK8
PIAS1
ZNF521
DEPTOR
HP
FLT3
NR1H3
ANXA3
ATP1B1
WT1
HOXB6
HOXA10
GATA2
SHANK3
DDAH2
MEX3B
MEIS1
GNAI1
RPA3
MYO5C
LCN2
CAMP
ELANE
HOXA7
MMP9
SPON1
LIN7A
DNMT3B
GNAI1
S100P
S1PR5
S1PR2
S1PR1
EZR
S1PR4
S1PR3
"

In [ ]:
genes <- read.table(text=m0.geneStr, sep="\n")
genes

In [ ]:
fmat <- mat[mat$name %in% genes$V1,]
dim(fmat)
head(fmat)

In [ ]:
rownames(fmat) <- fmat$name
fmat$X <- NULL
fmat$name <- NULL
dim(fmat)
head(fmat)

In [ ]:
flbl <- lbl
flbl <- flbl[flbl$FAB %in% c("CTRL", "M0"),]
flbl <- flbl[order(flbl$FAB),]
flbl <- flbl[-c(1:98),]
rownames(flbl) <- flbl$X
flbl$X.1 <- NULL
flbl$X <- NULL
dim(flbl)
head(flbl)

In [ ]:
flbl$FAB

In [ ]:
fmat <- fmat %>% select(row.names(flbl))
head(fmat)

In [ ]:
library(pheatmap)

In [ ]:
g <- pheatmap(
    fmat,
    labels_row=rownames(fmat),
    annotation_col=flbl,
    cellheight=4.5,
    cellwidth=4.5,
    width = 50,
    height = 50,
    cex=0.7
)

In [ ]:
save_pheatmap_png(g, "M0.png")

In [ ]:
library(limma)

In [ ]:
design <- model.matrix(~0+flbl$FAB)

In [ ]:
colnames(design) <- c("CTRL", "M0")
fit <- lmFit(fmat, design)
head(fit$coefficients)

In [ ]:
contrasts <- makeContrasts(M0 - CTRL, levels=design)
fit2 <- contrasts.fit(fit, contrasts)

In [ ]:
fit2 <- eBayes(fit2)

In [ ]:
topTable(fit2)

In [ ]:
results <- topTable(fit2, number=Inf)
results <- tibble::rownames_to_column(results, "name")

In [ ]:
library(ggplot2)
ggplot(results,aes(x = logFC, y=B)) + geom_point()

In [ ]:
upreg <- results[results$logFC > 2 & results$adj.P.Val < 0.05, 1]
downreg <- results[results$logFC < -2 & results$adj.P.Val < 0.05, 1]

In [ ]:
upreg <- str_replace_all(upreg, "\t", "\n")
writeLines(upreg, "m0-upreg.txt")
downreg <- str_replace_all(downreg, "\t", "\n")
writeLines(downreg, "m0-downreg.txt")

In [ ]:
filtered_results <- results[abs(results$logFC) > 2 & results$adj.P.Val < 0.05,]
fmat <- mat[mat$name %in% filtered_results$name,]
rownames(fmat) <- fmat$name
fmat$X <- NULL
fmat$name <- NULL
fmat <- fmat %>% select(row.names(flbl))
g <- pheatmap(
    fmat,
    labels_row=rownames(fmat),
    annotation_col=flbl,
    cellheight=4.5,
    cellwidth=4.5,
    width = 50,
    height = 50,
    cex=0.7
)
save_pheatmap_png(g, "M0.png")

# M1

In [ ]:
m1.geneStr = '"CEBPE"	"FUT1"	"CHIT1"	"SLPI"	"MMP9"	"MMP8"	"TGFBI"	"LTF"	"LCN2"	"CAMP"	"MMP26"	"CTSL"	"CEL"	"CTSB"	"PRSS2"	"COL4A3"	"COL4A2"	"MMP2"	"COL4A1"	"PRTN3"	"COL1A2"	"ELANE"	"COL2A1"	"COL1A1"	"MMP10"	"CYP4F2"	"FCAR"	"COL17A1"	"REPS1"	"DEPTOR"	"FLT3"	"HP"	"OLR1"	"NR1H3"	"WT1"	"CKAP4"	"CYSTM1"	"DUSP14"	"ORM1"	"LTF"	"LGALSL"	"CDA"	"SLC16A6"	"KLF5"	"DNM1"	"RCN1"	"MPG"	"QPCT"	"CCNB1IP1"	"SLPI"	"PLBD1"	"LCN2"	"CLTCL1"	"PFKP"	"MMP9"	"MMP8"	"EXT2"	"EGFR"	"LYN"	"NFKB1"	"MLST8"	"CD28"	"AGO2"	"IKBKG"	"RPS27A"	"IKBKB"	"NCK1"	"PLCG1"	"CALM3"	"CALM2"	"BTC"	"UBA52"	"CALM1"	"NFKBIB"	"PSME3"

'


In [ ]:
genes <- read.table(text=m1.geneStr, sep="\t")
genes <- t(genes)
genes

In [ ]:
fmat <- mat[mat$name %in% genes,]
dim(fmat)
head(fmat)

In [ ]:
rownames(fmat) <- fmat$name
fmat$X <- NULL
fmat$name <- NULL
dim(fmat)
head(fmat)

In [ ]:
flbl <- lbl
flbl <- flbl[flbl$FAB %in% c("CTRL", "M1"),]
flbl <- flbl[order(flbl$FAB),]
flbl <- flbl[-c(1:98),]
rownames(flbl) <- flbl$X
flbl$X.1 <- NULL
flbl$X <- NULL
dim(flbl)
head(flbl)

In [ ]:
flbl$FAB

In [ ]:
fmat <- fmat %>% select(row.names(flbl))
head(fmat)

In [ ]:
library(pheatmap)

In [ ]:
g <- pheatmap(
    fmat,
    labels_row=rownames(fmat),
    annotation_col=flbl,
    cellheight=4.5,
    cellwidth=4.5,
    width = 50,
    height = 50,
    cex=0.7
)

In [ ]:
save_pheatmap_png(g, "M1.png")

In [ ]:
library(limma)

In [ ]:
design <- model.matrix(~0+flbl$FAB)

In [ ]:
colnames(design) <- c("CTRL", "M1")
fit <- lmFit(fmat, design)
head(fit$coefficients)

In [ ]:
contrasts <- makeContrasts(M1 - CTRL, levels=design)
fit2 <- contrasts.fit(fit, contrasts)

In [ ]:
fit2 <- eBayes(fit2)

In [ ]:
topTable(fit2)

In [ ]:
results <- topTable(fit2, number=Inf)
results <- tibble::rownames_to_column(results, "name")

In [ ]:
library(ggplot2)
ggplot(results,aes(x = logFC, y=B)) + geom_point()

In [ ]:
upreg <- results[results$logFC > 2 & results$adj.P.Val < 0.05, 1]
downreg <- results[results$logFC < -2 & results$adj.P.Val < 0.05, 1]

In [ ]:
upreg <- str_replace_all(upreg, "\t", "\n")
writeLines(upreg, "m1-upreg.txt")
downreg <- str_replace_all(downreg, "\t", "\n")
writeLines(downreg, "m1-downreg.txt")

In [ ]:
filtered_results <- results[abs(results$logFC) > 2 & results$adj.P.Val < 0.05,]
fmat <- mat[mat$name %in% filtered_results$name,]
rownames(fmat) <- fmat$name
fmat$X <- NULL
fmat$name <- NULL
fmat <- fmat %>% select(row.names(flbl))
g <- pheatmap(
    fmat,
    labels_row=rownames(fmat),
    annotation_col=flbl,
    cellheight=4.5,
    cellwidth=4.5,
    width = 50,
    height = 50,
    cex=0.7
)
save_pheatmap_png(g, "M1.png")

# M2

In [ ]:
m2.geneStr = '"CHIT1"	"SLPI"	"MMP9"	"TGFBI"	"LCN2"	"CAMP"	"MMP26"	"CTSL"	"CTSB"	"PRSS2"	"COL4A3"	"COL4A2"	"MMP2"	"COL4A1"	"COL1A2"	"ELANE"	"COL2A1"	"PRTN3"	"COL1A1"	"MMP9"	"MMP10"	"MMP7"	"COL4A6"	"COL4A5"	"CTSS"	"COL4A4"	"KLF5"	"RPS19"	"RCN1"	"GLRX"	"CPXM1"	"JUP"	"SLC41A3"	"IRF4"	"FLT3"	"EGFL7"	"LCN2"	"PLBD1"	"CAMP"	"WT1"	"CYSTM1"	"DUSP14"	"KAT2A"	"MMP9"	"MEIS1"	"LIN7A"	"SLC16A6"	"ETS1"	"HIST2H2BE"	"NFKB1"	"HIST1H3J"	"FOS"	"HIST1H3I"	"CDKN2B"	"RPS27A"	"HIST1H3B"	"HIST1H3A"	"UBA52"	"HIST1H3D"	"HIST1H3C"	"HIST1H3F"	"HIST1H2BE"	"HIST1H3E"	"HIST1H3H"	"HIST1H3G"	"HIST2H4B"	"HIST2H4A"	"HIST1H4K"	"HIST1H4J"	"HIST1H4L"	"TP53"	"HIST1H4C"	"HIST1H4B"	"HIST1H4E"	"MAPK3"	"HIST1H4D"	"CEBPB"	"UBC"	"UBB"	"HIST1H4F"	"HIST1H4I"	"HIST1H4H"	"MAPK8"	"RAD50"	"HIST1H4A"	"CDK6"	"CDK2"	"MAP3K5"	"HIST1H1B"	"RELA"	"HIST2H2AC"	"MDM2"	"HIST2H3C"	"HIST4H4"	"HIST2H3D"	"CXCL8"	"HIST2H3A"
'

In [ ]:
genes <- read.table(text=m2.geneStr, sep="\t")
genes <- t(genes)
genes

In [ ]:
fmat <- mat[mat$name %in% genes,]
dim(fmat)
head(fmat)

In [ ]:
rownames(fmat) <- fmat$name
fmat$X <- NULL
fmat$name <- NULL
dim(fmat)
head(fmat)

In [ ]:
flbl <- lbl
flbl <- flbl[flbl$FAB %in% c("CTRL", "M2"),]
flbl <- flbl[order(flbl$FAB),]
flbl <- flbl[-c(1:98),]
rownames(flbl) <- flbl$X
flbl$X.1 <- NULL
flbl$X <- NULL
dim(flbl)
head(flbl)

In [ ]:
flbl$FAB

In [ ]:
fmat <- fmat %>% select(row.names(flbl))
head(fmat)

In [ ]:
library(pheatmap)

In [ ]:
g <- pheatmap(
    fmat,
    labels_row=rownames(fmat),
    annotation_col=flbl,
    cellheight=4.5,
    cellwidth=4.5,
    width = 50,
    height = 50,
    cex=0.7
)

In [ ]:
save_pheatmap_png(g, "M2.png")

In [ ]:
library(limma)

In [ ]:
design <- model.matrix(~0+flbl$FAB)

In [ ]:
colnames(design) <- c("CTRL", "M2")
fit <- lmFit(fmat, design)
head(fit$coefficients)

In [ ]:
contrasts <- makeContrasts(M2 - CTRL, levels=design)
fit2 <- contrasts.fit(fit, contrasts)

In [ ]:
fit2 <- eBayes(fit2)

In [ ]:
topTable(fit2)

In [ ]:
results <- topTable(fit2, number=Inf)
results <- tibble::rownames_to_column(results, "name")

In [ ]:
library(ggplot2)
ggplot(results,aes(x = logFC, y=B)) + geom_point()

In [ ]:
upreg <- results[results$logFC > 2 & results$adj.P.Val < 0.05, 1]
downreg <- results[results$logFC < -2 & results$adj.P.Val < 0.05, 1]

In [ ]:
upreg <- str_replace_all(upreg, "\t", "\n")
writeLines(upreg, "m2-upreg.txt")
downreg <- str_replace_all(downreg, "\t", "\n")
writeLines(downreg, "m2-downreg.txt")

In [ ]:
filtered_results <- results[abs(results$logFC) > 2 & results$adj.P.Val < 0.05,]
fmat <- mat[mat$name %in% filtered_results$name,]
rownames(fmat) <- fmat$name
fmat$X <- NULL
fmat$name <- NULL
fmat <- fmat %>% select(row.names(flbl))
g <- pheatmap(
    fmat,
    labels_row=rownames(fmat),
    annotation_col=flbl,
    cellheight=4.5,
    cellwidth=4.5,
    width = 50,
    height = 50,
    cex=0.7
)
save_pheatmap_png(g, "M2.png")

# M3

In [ ]:
m3.geneStr = '"NPW"	"GNA15"	"NPBWR1"	"OPRK1"	"OPRM1"	"NPBWR2"	"OPRD1"	"MMP2"	"PTGDS"	"MST1"	"HGF"	"LAMC1"	"ITGAM"	"COL23A1"	"SERPING1"	"TGFBI"	"EFNA1"	"KRT18"	"ADAM10"	"COL18A1"	"BCAN"	"COL17A1"	"LAMB1"	"FN1"	"TIMP1"	"TIMP2"	"DCN"	"COL4A3"	"FURIN"	"COL4A2"	"COL4A1"	"COL2A1"	"COL1A2"	"COL1A1"	"EXOC8"	"NID1"	"HSPG2"	"COL6A3"	"COL7A1"	"COL4A4"	"KLKB1"	"HPN"	"LAMC2"	"MMP14"	"LAMA5"	"PLG"	"ELANE"	"CASP3"	"CAPN1"	"A2M"	"ACAN"	"PALM"	"HOMER3"	"SLC35F2"	"SERPING1"	"ARHGAP4"	"WT1"	"KDM5B"	"CCNA1"	"LTF"	"ST3GAL4"	"EFNA1"	"DNM1"	"CFD"	"SMARCA1"	"TCN1"	"MXRA7"	"LAMC1"	"CABLES1"	"IFT22"	"FSCN1"	"KRT18"	"CD99"	"MMP2"	"PTGDS"	"CCND3"	"CTSW"	"GNA15"	"EXT2"	"NT5DC2"	"S100B"	"EGFR"	"ADAM10"	"PRKCA"	"NRAS"	"CDK1"	"PRKCE"	"PRKCD"	"HRAS"	"CREB1"	"PLCG1"	"SHC1"	"CALM3"	"CALM2"	"HGS"	"CALM1"	"FGFR1"	"CDC42"	"SRC"	"ITPR1"	"PRKACA"	"SH3GL2"	"CDKN1A"	"YWHAB"	"CDKN1B"	"CBL"	"EPS15"	"HSP90AA1"	"RAF1"	"ADRBK1"	"MDM2"	"PIK3R1"	"VAV1"	"GRB2"	"UBC"	"NR4A1"	"TSC2"	"AP2A2"	"AP2A1"

'


In [ ]:
genes <- read.table(text=m3.geneStr, sep="\t")
genes <- t(genes)
genes

In [ ]:
fmat <- mat[mat$name %in% genes,]
dim(fmat)
head(fmat)

In [ ]:
rownames(fmat) <- fmat$name
fmat$X <- NULL
fmat$name <- NULL
dim(fmat)
head(fmat)

In [ ]:
flbl <- lbl
flbl <- flbl[flbl$FAB %in% c("CTRL", "M3"),]
flbl <- flbl[order(flbl$FAB),]
flbl <- flbl[-c(1:98),]
rownames(flbl) <- flbl$X
flbl$X.1 <- NULL
flbl$X <- NULL
dim(flbl)
head(flbl)

In [ ]:
flbl$FAB

In [ ]:
fmat <- fmat %>% select(row.names(flbl))
head(fmat)

In [ ]:
library(pheatmap)

In [ ]:
g <- pheatmap(
    fmat,
    labels_row=rownames(fmat),
    annotation_col=flbl,
    cellheight=4.5,
    cellwidth=4.5,
    width = 50,
    height = 50,
    cex=0.7
)

In [ ]:
save_pheatmap_png(g, "M3.png")

In [ ]:
library(limma)

In [ ]:
design <- model.matrix(~0+flbl$FAB)

In [ ]:
colnames(design) <- c("CTRL", "M3")
fit <- lmFit(fmat, design)
head(fit$coefficients)

In [ ]:
contrasts <- makeContrasts(M3 - CTRL, levels=design)
fit2 <- contrasts.fit(fit, contrasts)

In [ ]:
fit2 <- eBayes(fit2)

In [ ]:
topTable(fit2)

In [ ]:
results <- topTable(fit2, number=Inf)
results <- tibble::rownames_to_column(results, "name")

In [ ]:
library(ggplot2)
ggplot(results,aes(x = logFC, y=B)) + geom_point()

In [ ]:
upreg <- results[results$logFC > 2 & results$adj.P.Val < 0.05, 1]
downreg <- results[results$logFC < -2 & results$adj.P.Val < 0.05, 1]

In [ ]:
upreg <- str_replace_all(upreg, "\t", "\n")
writeLines(upreg, "m3-upreg.txt")
downreg <- str_replace_all(downreg, "\t", "\n")
writeLines(downreg, "m3-downreg.txt")

In [ ]:
filtered_results <- results[abs(results$logFC) > 2 & results$adj.P.Val < 0.05,]
fmat <- mat[mat$name %in% filtered_results$name,]
rownames(fmat) <- fmat$name
fmat$X <- NULL
fmat$name <- NULL
fmat <- fmat %>% select(row.names(flbl))
g <- pheatmap(
    fmat,
    labels_row=rownames(fmat),
    annotation_col=flbl,
    cellheight=4.5,
    cellwidth=4.5,
    width = 50,
    height = 50,
    cex=0.7
)
save_pheatmap_png(g, "M3.png")

# M4

In [ ]:
m4.geneStr = '"COL17A1"	"PKP2"	"KRT18"	"ITGA6"	"COL17A1"	"DST"	"KRT5"	"PLEC"	"ITGB4" 	"COL17A1"	"PKP2"	"KRT18"	"DSP"	"DSC2"	"DSG2"	"JUP"	"PKP2"	"HOMER3"	"COL17A1"	"CD8A"	"HP"	"FLT3"	"TACSTD2"	"TNS3"	"TIMP1"	"ANXA3"	"SUSD3"	"WT1"	"IL18RAP"	"PRTN3"	"HOXA10"	"HOXB7"	"MEIS1"	"OLFM4"	"KLF5"	"PLXNB2"	"SYNE1"	"LRIG1"	"PKP2"	"KRT18"	"CAMP"	"ACY3"	"CEACAM6"	"MMP9"	"ETS1"	"CDK1"	"CREBBP"	"FOS"	"SMURF2"	"FN1"	"CDKN2B"	"ATF2"	"COPS5"	"UBE2I"	"SHC1"	"HGS"	"COL1A2"	"MET"	"CDC42"	"THBS1"	"YAP1"	"SMAD3"	"EP300"	"APP"	"SUMO1"	"SRC"	"BCAR1"	"ITGB1"	"ITGB4"	"SIN3A"	"TP53"	"RAF1"	"PIK3R1"	"MMP1"	"GRB2"	"JUND"	"WWP1"	"TGFB1"	"MAPK14"	"PIAS2"	"MAPK8"	"HDAC1"	"MAPK9"	"NEDD4L"	"PTK2"
'

In [ ]:
genes <- read.table(text=m4.geneStr, sep="\t")
genes <- t(genes)
genes

In [ ]:
fmat <- mat[mat$name %in% genes,]
dim(fmat)
head(fmat)

In [ ]:
rownames(fmat) <- fmat$name
fmat$X <- NULL
fmat$name <- NULL
dim(fmat)
head(fmat)

In [ ]:
flbl <- lbl
flbl <- flbl[flbl$FAB %in% c("CTRL", "M4"),]
flbl <- flbl[order(flbl$FAB),]
flbl <- flbl[-c(1:98),]
rownames(flbl) <- flbl$X
flbl$X.1 <- NULL
flbl$X <- NULL
dim(flbl)
head(flbl)

In [ ]:
flbl$FAB

In [ ]:
fmat <- fmat %>% select(row.names(flbl))
head(fmat)

In [ ]:
library(pheatmap)

In [ ]:
g <- pheatmap(
    fmat,
    labels_row=rownames(fmat),
    annotation_col=flbl,
    cellheight=4.5,
    cellwidth=4.5,
    width = 50,
    height = 50,
    cex=0.7
)

In [ ]:
save_pheatmap_png(g, "M4.png")

In [ ]:
library(limma)

In [ ]:
design <- model.matrix(~0+flbl$FAB)

In [ ]:
colnames(design) <- c("CTRL", "M4")
fit <- lmFit(fmat, design)
head(fit$coefficients)

In [ ]:
contrasts <- makeContrasts(M4 - CTRL, levels=design)
fit2 <- contrasts.fit(fit, contrasts)

In [ ]:
fit2 <- eBayes(fit2)

In [ ]:
topTable(fit2)

In [ ]:
results <- topTable(fit2, number=Inf)
results <- tibble::rownames_to_column(results, "name")

In [ ]:
library(ggplot2)
ggplot(results,aes(x = logFC, y=B)) + geom_point()

In [ ]:
upreg <- results[results$logFC > 2 & results$adj.P.Val < 0.05, 1]
downreg <- results[results$logFC < -2 & results$adj.P.Val < 0.05, 1]

In [ ]:
upreg <- str_replace_all(upreg, "\t", "\n")
writeLines(upreg, "m4-upreg.txt")
downreg <- str_replace_all(downreg, "\t", "\n")
writeLines(downreg, "m4-downreg.txt")

In [ ]:
filtered_results <- results[abs(results$logFC) > 2 & results$adj.P.Val < 0.05,]
fmat <- mat[mat$name %in% filtered_results$name,]
rownames(fmat) <- fmat$name
fmat$X <- NULL
fmat$name <- NULL
fmat <- fmat %>% select(row.names(flbl))
g <- pheatmap(
    fmat,
    labels_row=rownames(fmat),
    annotation_col=flbl,
    cellheight=4.5,
    cellwidth=4.5,
    width = 50,
    height = 50,
    cex=0.7
)
save_pheatmap_png(g, "M4.png")

# M5

In [ ]:
m5.geneStr = '"PRTN3"	"PKIB"	"IL1B"	"LY6D"	"CTTN"	"APMAP"	"FLT3"	"CD24"	"CD59"	"BMX"	"LCN2"	"TNS3"	"CAMP"	"CEBPE"	"PRTN3"	"CEACAM1"	"HOXA5"	"HOXB6"	"HOXA10"	"HOXB7"	"HOXA7"	"PLK2"	"MMP9"	"KLF1"	"MEIS1"	"LYN"	"JUN"	"PTPN11"	"F2RL1"	"PTPN6"	"SPI1"	"PRTN3"	"HOXB7"	"GATA1"	"ANXA2"	"MEIS1"	"BATF"	"FLI1"	"EP300"	"RB1"	"SRC"	"TNF"	"JUNB"	"CEBPG"	"MYB"	"TGFB1"	"CTTN"	"USP40"	"CTDSPL"	"FLT3"	"HP"	"BMX"	"TNS3"	"SFXN3"	"OLR1"	"ARG1"	"ANXA3"	"ACSF2"	"HOXA10"	"ORM1"	"ARHGEF40"	"LTF"	"MEIS1"	"AMPD3"	"APMAP"	"TCN1"	"PKIB"	"RGL4"	"CD59"	"STS"	"CEBPE"	"PLA2G15"	"CEACAM1"	"HOXA5"	"HOXA6"	"HOXA7"	"PLK2"	"MMP9"	"CPNE8"	"KLF1"
'

In [ ]:
genes <- read.table(text=m5.geneStr, sep="\t")
genes <- t(genes)
genes

In [ ]:
fmat <- mat[mat$name %in% genes,]
dim(fmat)
head(fmat)

In [ ]:
rownames(fmat) <- fmat$name
fmat$X <- NULL
fmat$name <- NULL
dim(fmat)
head(fmat)

In [ ]:
flbl <- lbl
flbl <- flbl[flbl$FAB %in% c("CTRL", "M5"),]
flbl <- flbl[order(flbl$FAB),]
flbl <- flbl[-c(1:98),]
rownames(flbl) <- flbl$X
flbl$X.1 <- NULL
flbl$X <- NULL
dim(flbl)
head(flbl)

In [ ]:
flbl$FAB

In [ ]:
fmat <- fmat %>% select(row.names(flbl))
head(fmat)

In [ ]:
library(pheatmap)

In [ ]:
g <- pheatmap(
    fmat,
    labels_row=rownames(fmat),
    annotation_col=flbl,
    cellheight=4.5,
    cellwidth=4.5,
    width = 50,
    height = 50,
    cex=0.7
)

In [ ]:
save_pheatmap_png(g, "M5.png")

In [ ]:
library(limma)

In [ ]:
design <- model.matrix(~0+flbl$FAB)

In [ ]:
colnames(design) <- c("CTRL", "M5")
fit <- lmFit(fmat, design)
head(fit$coefficients)

In [ ]:
contrasts <- makeContrasts(M5 - CTRL, levels=design)
fit2 <- contrasts.fit(fit, contrasts)

In [ ]:
fit2 <- eBayes(fit2)

In [ ]:
topTable(fit2)

In [ ]:
results <- topTable(fit2, number=Inf)
results <- tibble::rownames_to_column(results, "name")

In [ ]:
library(ggplot2)
ggplot(results,aes(x = logFC, y=B)) + geom_point()

In [ ]:
upreg <- results[results$logFC > 2 & results$adj.P.Val < 0.05, 1]
downreg <- results[results$logFC < -2 & results$adj.P.Val < 0.05, 1]

In [ ]:
upreg <- str_replace_all(upreg, "\t", "\n")
writeLines(upreg, "m5-upreg.txt")
downreg <- str_replace_all(downreg, "\t", "\n")
writeLines(downreg, "m5-downreg.txt")

In [ ]:
filtered_results <- results[abs(results$logFC) > 1 & results$adj.P.Val < 0.05,]
fmat <- mat[mat$name %in% filtered_results$name,]
rownames(fmat) <- fmat$name
fmat$X <- NULL
fmat$name <- NULL
fmat <- fmat %>% select(row.names(flbl))
g <- pheatmap(
    fmat,
    labels_row=rownames(fmat),
    annotation_col=flbl,
    cellheight=4.5,
    cellwidth=4.5,
    width = 50,
    height = 50,
    cex=0.7
)
save_pheatmap_png(g, "M5.png")